Import libraries 

In [1]:
import torch
import torch.nn as nn
from torchtext import data
from torchtext import datasets
import numpy as np
import pandas as pd

from torchtext.vocab import GloVe

Here we will import our own dataset: a personality prediction using mypersonlity dataset<br />
which is unfortunately not available anymore (see their statement: https://sites.google.com/michalkosinski.com/mypersonality)

In [3]:
# Load prediction target, which are the big5 personaity scores
y_AG = []
y_CO = []
y_ES = []
y_EX = []
y_IT = []
with open("./wv_myp_observation.txt", encoding="UTF-8") as obFile:
    for line in obFile:
        info = line.strip().split(",")
        y_AG.append([info[1]])
        y_CO.append([info[2]])
        y_ES.append([info[3]])
        y_EX.append([info[4]])
        y_IT.append([info[5]])
y_AG = np.array(y_AG, dtype="float32")
y_CO = np.array(y_CO, dtype="float32")
y_ES = np.array(y_ES, dtype="float32")
y_EX = np.array(y_EX, dtype="float32")
y_IT = np.array(y_IT, dtype="float32")

In [4]:
# Select target
y = y_AG

In [5]:
# Load the observation, which are Facebook posts
dat = []
with open("./wv_myp_allposts.txt", encoding="UTF-8") as obFile:
    for line in obFile:
        dat.append(line.strip())

In [6]:
allData = pd.concat((pd.DataFrame(dat), pd.DataFrame(y)), axis=1)
allData.columns = ['text', 'big5']
# Save the data to csv
allData.to_csv("./allData_personality.csv", sep=';', encoding='utf-8')
# Print the first row
allData.iloc[0]

text    last night was amazing not only did see propna...
big5                                                 0.65
Name: 0, dtype: object

In [7]:
# Do a 10-fold CV, here we only perform the first fold for simplicity
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)
for train_index, test_index in kf.split(allData.index):
    print(train_index)
    print(test_index)
    break

[  0   1   2   4   5   6   8   9  10  11  12  13  14  15  17  18  19  20
  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  39
  40  41  42  43  44  46  47  48  49  50  51  52  53  54  55  56  57  59
  60  61  62  63  64  65  66  67  68  72  73  74  75  76  77  78  79  80
  81  82  83  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136 137 138
 139 141 142 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 175 176 177
 179 180 181 182 183 184 186 187 188 190 191 192 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 231 232 233 234 235 236 237
 239 240 241 242 243 244 245 246 248]
[  3   7  16  38  45  58  69  70  71  84  85 105 133 140 143 174 178 185
 189 193 213 

In [8]:
testData = allData.iloc[test_index]
testData.to_csv("./testData_personality.csv", sep=';', encoding='utf-8')
trainData = allData.iloc[train_index]
trainData.to_csv("./trainData_personality.csv", sep=';', encoding='utf-8')

Split the dataset to train, test, and val

In [9]:
# Load dataset from csv
df_train = pd.read_csv("./trainData_personality.csv", encoding="UTF-8", delimiter=";")

# Set random seed and shuffle the data
idx = np.arange(df_train.shape[0])
np.random.seed(999)
np.random.shuffle(idx)

# Set the size of validation set
VAL_RATIO = 0.15
val_size = int(len(idx) * VAL_RATIO)

# Split dataset to val and train dataset
df_train.iloc[idx[val_size:], :].to_csv(
    "cache/dataset_train2.csv", index=False)
df_train.iloc[idx[:val_size], :].to_csv(
    "cache/dataset_val2.csv", index=False)

# Read the testing dataset
df_test = pd.read_csv("./testData_personality.csv", encoding="UTF-8", delimiter=";")
df_test.to_csv("cache/dataset_test2.csv", index=False)
print(df_test)

    Unnamed: 0                                               text   big5
0            3  thanks for all the birthday and christmas wish...  0.650
1            7  laundry then packing then maybe sleeping ahhhh...  1.000
2           16  bout sleep goodnight for tonight yoooo that ju...  0.820
3           38  meh about open source because windows random e...  0.760
4           45  just remembered seen naked guy running the sup...  0.620
5           58  got his permit has the flu and under quarantin...  0.560
6           69  thinks this the best day ever propname 30th bi...  0.750
7           70  miss sister thank you everyone for the birthda...  0.634
8           71  wondering work out actually working i'm kickin...  0.950
9           84  its way early saturday heading work relaxing c...  0.800
10          85  back reality some phone from san ramon called ...  0.800
11         105  going flordia tomorrow vacation flordia for we...  0.500
12         133  youtube rules honestly does green d

Specify all fields 

In [10]:
# sequential = True: specify that the data is sequential
# include_lengths = True: a list that store the length of each sentence will also be returned
# lower = True: lowercase all words
# tokenize = tokenizer: use out pre-defined tokenizer to tokenize the text. Default is text.split()
# init_token = "<SOS>": a <SOS> will be added before the beginning of the text sequence
# eos_token = "<EOS>": a <EOS> will be added after the end of the text sequence
# We did not specify a parameter "use_vocab" cause it's already set to True, 
#        this parameter specify that all words in the text sequence will be converted into indexes
#        and we can see the corresponding words to the indexes using TEXT.vocab.stoi
TEXT = data.Field(sequential=True, include_lengths=True, lower=True, init_token='<SOS>', eos_token='<EOS>')

# Label is our predict target, which is numeric 
# We specify the tensor type of this field shoud be float, not sequential, and shouldn't be convert to word indexes
LABEL = data.Field(sequential=False, use_vocab=False, tensor_type=torch.FloatTensor)

# Load the datasets
# Discard the "Unnamed: 0" column by assigning it to None
train,test, val = data.TabularDataset.splits(
    path='cache/', format='csv', skip_header=True,
    train='dataset_train2.csv', validation='dataset_val2.csv', test="dataset_test2.csv",
    fields=[
        ('Unnamed: 0', None),
        ('text', TEXT),
        ('big5', LABEL)
    ])

Here we print out an Example in the training dataset.<br />
You might notice that the data we get is a torchtext.data.example.Example object.<br />
To print out the attribute of this object, use ".attribute"

In [11]:
for num, batch in enumerate(train):
    print(num)
    print(batch)
    print(batch.text)
    print(batch.big5)
    break

0
['watching', 'gossip', 'girl', 'this', 'may', 'mean', "i've", 'failed', 'acquire', 'taste', 'died', 'wants', 'sleep', 'going', 'the', 'lone', 'star', 'ruby', 'conf', 'august', 'salvador', "dali's", 'distorted', 'clocks', 'were', 'inspired', 'melting', 'cheese', "i'll", 'never', 'eat', 'grilled', 'cheese', 'sandwich', 'the', 'same', 'way', 'again', 'find', 'funny', 'that', 'keep', 'clicking', 'unfamiliar', 'names', 'and', 'wondering', 'when', 'did', 'friend', 'this', 'person', 'and', 'until', 'realize', 'they', 'got', 'married', 'thinks', 'ice', 'the', 'best', 'puppy', 'treat', 'ever', 'puppy', 'training', 'classes', 'then', 'uga', 'football', 'weee', 'wine', 'cheese', "teebow's", 'tears', 'fun', 'enjoying', 'the', 'latest', 'imogen', 'heap', 'album', 'atlanta', 'spending', 'night', 'with', 'ibm', 'programmers', 'feels', 'like', 'chapter', 'out', 'microserfs', 'will', 'miss', 'stadler', 'heart', 'ruby', 'made', 'spicy', 'coconut', 'soup', 'mmm', 'back', 'austin', 'just', 'watched', 'b

To convert words to indexes, we need to use the build_vocab() method from torchtext.data.Field object.

In [12]:
# Use the in-built word embedding model GloVe
# You can use your own word embedding model using torchtext.vocab.Vectors
TEXT.build_vocab(train, test, val, vectors="glove.6B.300d", max_size=30000) 


In [13]:
# The converted word embeddings are stored under TEXT.vocab.vectors
TEXT.vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [14]:
# The index-word correpondance is recorded in TEXT.vocab.stoi as a dictionary
# We build a lookup dictionary for later use
lookup = {b:a for a, b in TEXT.vocab.stoi.items()}
lookup[30]

'from'

Split the dataset into batches

In [15]:
# batch_size = 32: In each batch contains 32 instaces. 
#       It has been proved that training in a batch gives better performance than learn a single data at a time
# device = torch.device("cuda"): Use GPU. If you do not have a GPU enviornment, use torch.device("cpu") instead
# repeat = False: Do not repeat the iterator for multiple epochs
# sort_key = lambda...: A key to use for sorting examples in order to batch together examples with similar lengths 
#       and minimize padding. The sort_key provided to the Iterator constructor overrides the sort_key attribute of  
#       the Dataset, or defers to it if None
train_iter, test_iter, val_iter = data.BucketIterator.splits(
          (train, test, val), batch_size=32, device=torch.device("cuda"), repeat=False, sort_key=lambda x: len(x.text))

Here we print out the data in train_iter

In [16]:
for num, batch in enumerate(train_iter):
    # batch is a torchtext.data.batch.Batch object
    # In batch there are 32 instances
    # The data in batch have two fields: text and label
    # The text field contains the text index itself [len(longest sentence)x32] and 
    #          the length of each setence [32] in this batch (remember include_lengths=True?)
    # The label field contains the prediction target [32]
    print("batch:")
    print(batch, "\n")
    
    # Print the label
    print("batch.big5:")
    print(batch.big5, "\n")
    
    # Print the first component of batch.text (the sequence of words)
    print("batch.text[0]:")
    print(batch.text[0], "\n")
    
    # Print the first element in batch.text[0], which is all first words in all the sentences
    # You may notice that the results are all 2
    # That's because we use <SOS> to represent the beginning of a sentence
    # Going back to TEXT.vocab.stoi you can find that the index of <SOS> is 2
    print("batch.text[0][0]:")
    print(batch.text[0][0], "\n")
    
    # Print the actual content fifth sentence in this batch
    # Noted that we use the lookup dictionary to recover the word index to word
    print("Fifth sentence: ")
    for i in range(batch.text[0].size()[0]):
        print(lookup[batch.text[0][i].tolist()[5]], end=" ")
    # You might find that torchtext automatically add <PAD>s after <EOS> to make all sentence 
    #           having identical length in this batch
    break

batch:

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 953x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.big5]:[torch.cuda.FloatTensor of size 32 (GPU 0)] 

batch.big5:
tensor([ 0.6000,  0.9500,  0.8000,  0.6000,  0.7400,  0.6500,  0.6000,
         0.5600,  0.7900,  0.8100,  0.8000,  0.4600,  0.7700,  0.8700,
         0.7700,  0.6600,  0.6000,  0.6500,  0.8760,  0.6700,  0.4000,
         0.6000,  0.8500,  0.6700,  0.5000,  0.7000,  0.7500,  0.7500,
         0.9100,  0.8300,  0.8600,  0.7900], device='cuda:0') 

batch.text[0]:
tensor([[     2,      2,      2,  ...,      2,      2,      2],
        [  5036,   1612,    125,  ...,    613,    158,     46],
        [ 11954,    361,   1640,  ...,     21,     24,     31],
        ...,
        [     1,      1,      1,  ...,      1,      1,      1],
        [     1,      1,      1,  ...,      1,      1,      1],
        [     1,      1,      1,  ...,      1,      1,      1]], device='cuda:0') 


Finally, copy the word embeddings to torch.nn.Embedding object
Then you can use this embed for your RNN model! :)

In [17]:
# The second parameter(300) is the number of dimension of your embedding model
# padding_idx = 1: specify the <PAD>'s index is 1
# max_norm = 1: If given, will renormalize the embeddings to always have a norm lesser than the given number
embed = nn.Embedding(len(TEXT.vocab), 300, padding_idx=1, max_norm=1)

# Copy the word vectors from dataset to nn.Embedding object
embed.weight.data.copy_(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])